In [1]:

##### ML implementation of CF (Fully Works)


from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.sql import Row

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pandas.io import gbq

"""from google.cloud import bigquery
from google.cloud.bigquery import SchemaField"""


'from google.cloud import bigquery\nfrom google.cloud.bigquery import SchemaField'

In [3]:
query = """SELECT * FROM [lyngro-1191:ml.sample_user_post_2]
  limit 2000 """
data = gbq.read_gbq(query=query, 
                     dialect ='legacy', project_id='lyngro-1191')
data.head()
data.describe()

Requesting query... ok.
Query running...
Query done.
Processed: 28.7 Gb

Retrieving results...
Got 1800 rows.

Total time taken 4.89 s.
Finished at 2017-06-15 15:40:54.


,userid,postid
count,1800,1800
unique,30,983
top,e3b86653-902b-421a-a03f-2c665891c56c,3fa11a20-ded0-4298-8540-a721009079e6
freq,60,60


In [4]:
ratings = np.ones(len(data), dtype=np.int)

print len(ratings)

1800


In [5]:
x = pd.unique(data.values[:,0].ravel())
x = pd.Series(np.arange(len(x)), x)
y = pd.unique(data.values[:,1].ravel())
y = pd.Series(np.arange(len(y)), y)


datax = data[['userid']].applymap(x.get)
datay = data[['postid']].applymap(y.get)
#print datax.describe()


data["nuserid"] = datax
data["npostid"] = datay
data["rating"] = ratings

new_data = data[['nuserid', 'npostid','rating']]

#data.sort_values('npostid')

data.describe()

,nuserid,npostid,rating
count,1800.000000,1800.000000,1800.0
mean,14.500000,419.813333,1.0
std,8.657847,265.804301,0.0
min,0.000000,0.000000,1.0
25%,7.000000,209.000000,1.0
50%,14.500000,389.000000,1.0
75%,22.000000,623.250000,1.0
max,29.000000,982.000000,1.0


In [7]:
new_data

,nuserid,npostid,rating
0,0,0,1
1,0,1,1
2,0,2,1
3,0,3,1
4,0,4,1
5,0,5,1
6,0,6,1
7,0,7,1
8,0,8,1
9,0,9,1


In [6]:
data_sdf = spark.createDataFrame(new_data) #convert from pandas dataframe to spark dataframe

print data_sdf.describe()

#print data_sdf.toPandas().sort("npostid")

#print data_sdf.toPandas().head()


#print new_data.describe()

DataFrame[summary: string, nuserid: string, npostid: string, rating: string]


In [7]:
(training, validation, test) = data_sdf.randomSplit([0.6, 0.2, 0.2])

#(training2, test2) = spark.createDataFrame(data).randomSplit([0.8, 0.2])


#print training.toPandas()
#print test.toPandas()

#test.show()




In [36]:
def computeRmse(model, data, n):
    """
    Compute RMSE (Root Mean Squared Error).
    """
    predictions = model.predictAll(data.map(lambda x: (x[0], x[1])))
    predictionsAndRatings = predictions.map(lambda x: ((x[0], x[1]), x[2])) \
      .join(data.map(lambda x: ((x[0], x[1]), x[2]))) \
      .values()
    return sqrt(predictionsAndRatings.map(lambda x: (x[0] - x[1]) ** 2).reduce(add) / float(n))

In [ ]:
import itertools

ranks = [8, 12]
lambdas = [0.1, 10.0]
numIters = [10, 20]
bestModel = None
bestValidationRmse = float("inf")
bestRank = 0
bestLambda = -1.0
bestNumIter = -1

als = ALS(rank=10, maxIter=5, regParam=0.01, alpha=1, implicitPrefs=True, userCol="nuserid", itemCol="npostid", ratingCol="rating")


for rank, lmbda, numIter in itertools.product(ranks, lambdas, numIters):
    model = ALS.fit(training, params=[])
    validationRmse = computeRmse(model, validation, numValidation)
    print "RMSE (validation) = %f for the model trained with " % validationRmse + \
              "rank = %d, lambda = %.1f, and numIter = %d." % (rank, lmbda, numIter)
    if (validationRmse < bestValidationRmse):
        bestModel = model
        bestValidationRmse = validationRmse
        bestRank = rank
        bestLambda = lmbda
        bestNumIter = numIter

testRmse = computeRmse(bestModel, test, numTest)


In [9]:
import itertools

ranks = [8, 12]
lambdas = [0.1, 10.0]
numIters = [10, 20]
alphas = 1
bestModel = None
bestValidationRmse = float("inf")
bestRank = 0
bestLambda = -1.0
bestNumIter = -1

als = ALS(rank=10, maxIter=5, regParam=0.01, alpha=1, implicitPrefs=True, userCol="nuserid", itemCol="npostid", ratingCol="rating")


ranks1 = 12
numIters1 = 20
als = als.setParams(rank=ranks, maxIter=numIters)
### The following statement doesn not work. ALS must be initiailised with specific values
#als = ALS(rank=ranks, maxIter=numIters, regParam=lambdas, alpha=alphas, implicitPrefs=True, userCol="nuserid", itemCol="npostid", ratingCol="rating")

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

for rank, lmbda, numIter in itertools.product(ranks, lambdas, numIters):
    model = als.fit(training)
    predictions = model.transform(validation)
    validationRmse = evaluator.evaluate(predictions)
    print("Root-mean-square error = " + str(rmse))

    #validationRmse = computeRmse(model, validation, numValidation)
    print "RMSE (validation) = %f for the model trained with " % validationRmse + \
              "rank = %d, lambda = %.1f, and numIter = %d." % (rank, lmbda, numIter)
    if (validationRmse < bestValidationRmse):
        bestModel = model
        bestValidationRmse = validationRmse
        bestRank = rank
        bestLambda = lmbda
        bestNumIter = numIter

testPredictions = bestModel.transform(test)
bestRMSE = evaluator.evaluate(testPredictions)


TypeError: Invalid param value given for param "rank". Could not convert [8, 12] to int

In [8]:
# Build the recommendation model using ALS on the training data


# Implicit ALS
als = ALS(rank=10, maxIter=5, regParam=0.01, alpha=1, implicitPrefs=True, userCol="nuserid", itemCol="npostid", ratingCol="rating")

# paramter recently added: coldStartStrategy="drop"

# Explicit ALS
#als = ALS(maxIter=5, regParam=0.01, alpha=1, userCol="nuserid", itemCol="npostid", ratingCol="rating")

model = als.fit(training)
#model.setColdStartStrategy("drop")

#model_df = model.rdd
#print spark.createDataFrame(model).show()

In [9]:

# prints the model userFactors/itemFactors sorted by id - both have 10 features
#print model.userFactors.orderBy("id").collect()
#print model.itemFactors.orderBy("id").collect()

new_model = model.transform(test)

new_model = new_model.drop()

predictions = sorted(new_model.collect(), key=lambda r: r[0])

model_pd = new_model.toPandas()

model_pd2 = model_pd.dropna()

predictions = spark.createDataFrame(model_pd2)

predictions.show()


+-------+-------+------+--------------------+
|nuserid|npostid|rating|          prediction|
+-------+-------+------+--------------------+
|     24|    148|     1|-0.03886827081441879|
|     24|    148|     1|-0.03886827081441879|
|     20|    243|     1|-0.02907812222838...|
|     19|    243|     1|-0.20556847751140594|
|     13|    516|     1| 0.22902901470661163|
|      1|     85|     1| 0.25729286670684814|
|      2|    137|     1| -0.0767446756362915|
|      7|    251|     1| 0.31363290548324585|
|     11|    451|     1| 0.22892840206623077|
|     15|    588|     1|   0.668765127658844|
|      2|    133|     1|0.032676272094249725|
|     23|     78|     1|  0.9354592561721802|
|     19|    385|     1|  0.2095988243818283|
|     29|    183|     1|  0.4168221354484558|
|     21|    183|     1|-0.19415771961212158|
|      4|    210|     1| 0.44138237833976746|
|     10|    436|     1|  0.5036879181861877|
|     15|    406|     1|-0.07890857756137848|
|     26|    731|     1|  1.554702

In [41]:
from pyspark import since, keyword_only
from pyspark.ml.util import *
from pyspark.ml.wrapper import JavaEstimator, JavaModel
from pyspark.ml.param.shared import *
from pyspark.ml.common import inherit_doc

temp_path = "/Users/mo/Documents/"

als_path = temp_path + "/als"
als.save(als_path)
als2 = ALS.load(als_path)
print als.getMaxIter()



model_path = temp_path + "/als_model"
model.save(model_path)
model2 = ALSModel.load(model_path) #ALSModel
model.rank == model2.rank

print sorted(model.userFactors.collect()) == sorted(model2.userFactors.collect())

print sorted(model.itemFactors.collect()) == sorted(model2.itemFactors.collect())

predictions = sorted(model.transform(test).collect(), key=lambda r: r[0])
predictions[0]

print "The full list"

print predictions


5
True
True
The full list
[Row(nuserid=0, npostid=52, rating=1.0, prediction=-0.05828510597348213), Row(nuserid=0, npostid=13, rating=1.0, prediction=nan), Row(nuserid=0, npostid=19, rating=1.0, prediction=0.19138935208320618), Row(nuserid=0, npostid=9, rating=1.0, prediction=0.43272656202316284), Row(nuserid=0, npostid=35, rating=1.0, prediction=0.2624175548553467), Row(nuserid=0, npostid=23, rating=1.0, prediction=0.15798233449459076), Row(nuserid=0, npostid=50, rating=1.0, prediction=0.017242439091205597), Row(nuserid=0, npostid=32, rating=1.0, prediction=0.17229697108268738), Row(nuserid=0, npostid=33, rating=1.0, prediction=0.34538954496383667), Row(nuserid=0, npostid=14, rating=1.0, prediction=0.6389124393463135), Row(nuserid=1, npostid=85, rating=1.0, prediction=0.28326350450515747), Row(nuserid=1, npostid=76, rating=1.0, prediction=0.5603967905044556), Row(nuserid=1, npostid=97, rating=1.0, prediction=0.048073433339595795), Row(nuserid=1, npostid=77, rating=1.0, prediction=0.17

In [10]:
##### Evaluate the model by computing the RMSE on the test data
#predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))


Root-mean-square error = 0.815177608991
